# Title: Aspect_Ratio_Preserving_Augmentation_Pipeline
## Description: 이미지의 가로세로 비율을 유지하며 정사각형으로 패딩(Padding)한 후 리사이즈하는 전처리 기법과, 일반화 성능을 높이는 GridDropout 등의 증강 파이프라인.
## Input: 
 - image (numpy array): OpenCV로 로드된 이미지 (H, W, C)
 - cfg (dict): 이미지 사이즈(IMG_SIZE) 등이 정의된 설정 딕셔너리
## Output: 
 - augmented_image (tensor): 모델 입력용 정규화 및 증강된 텐서
## Check Point: 
 - `albumentations` 라이브러리 필요.
 - `PadSquare`는 `ImageOnlyTransform`을 상속받아 커스텀 구현됨.

 ![패딩 예시](/workspaces/my_DS_recipe_book/02_Preprocessing/image.png)

In [ ]:
import cv2
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from albumentations.core.transforms_interface import ImageOnlyTransform

# [Block 1] Custom Transform: 비율 유지 패딩 (PadSquare)
class PadSquare(ImageOnlyTransform):
    """
    이미지를 리사이즈하기 전, 원본 비율을 유지하기 위해
    짧은 변을 기준으로 부족한 영역을 검은색(0)으로 채워 정사각형을 만드는 클래스
    """
    def __init__(self, border_mode=0, value=0, always_apply=False, p=1.0):
        super().__init__(always_apply, p)
        self.border_mode = border_mode
        self.value = value

    def apply(self, image, **params):
        h, w, c = image.shape
        max_dim = max(h, w)
        
        # 정사각형을 만들기 위해 필요한 패딩 계산 ( 각 방향별로 균등하게 분배 )
        pad_h = max_dim - h
        pad_w = max_dim - w

        top = pad_h // 2
        bottom = pad_h - top
        left = pad_w // 2
        right = pad_w - left

        # cv2.copyMakeBorder를 이용해 상하좌우 패딩 적용
        image = cv2.copyMakeBorder(
            image, top, bottom, left, right,
            cv2.BORDER_CONSTANT, value=self.value
        )
        return image

    def get_transform_init_args_names(self):
        return ('border_mode', 'value')

# [Block 2] Augmentation Pipeline Definition
def get_transforms(mode='train', img_size=224):
    """
    학습 및 추론용 증강 파이프라인 생성
    """
    if mode == 'train':
        return A.Compose([
            # 1. 비율 유지 패딩 (가장 먼저 적용)
            PadSquare(value=(0, 0, 0)),
            
            # 2. Scale Invariance 학습을 위한 Random Crop
            # 작은 물체도 학습할 수 있도록 다양한 스케일로 자른 후 리사이즈
            A.RandomResizedCrop(
                size=(img_size, img_size), 
                scale=(0.3, 1.0), 
                ratio=(0.9, 1.1), 
                p=1.0
            ),
            
            # 3. 기하학적 변환 (Affine, Rotate, Flip)
            A.Affine(scale=(0.9, 1.1), translate_percent=(0.05, 0.05), shear=(-5, 5), p=0.5),
            A.RandomRotate90(p=0.5),
            A.HorizontalFlip(p=0.5),
            
            # 4. 픽셀값 변환 (Brightness)
            A.RandomBrightnessContrast(limit=0.15, p=0.5),
            
            # 5. 정규화 (ImageNet 통계량)
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            
            # 6. GridDropout: 물체의 일부분을 가려 강건함(Robustness) 향상
            A.GridDropout(ratio=0.2, unit_size_min=5, p=0.5),
            
            ToTensorV2()
        ])
    
    elif mode == 'test':
        return A.Compose([
            PadSquare(value=(0, 0, 0)), # 테스트 시에도 동일하게 비율 유지
            A.Resize(img_size, img_size),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ToTensorV2()
        ])

## How to Use
1. **파이프라인 생성**: `train_transform = get_transforms(mode='train', img_size=224)` 형태로 호출하여 사용합니다.
2. **Dataset 적용**: PyTorch `Dataset` 클래스의 `__getitem__` 메서드 내에서 다음과 같이 적용합니다.
    ```python
    if self.transforms:
        image = self.transforms(image=image)['image']
    ```

## Troubleshooting
- **패딩 색상(value)**: 기본값은 검은색 `(0,0,0)`입니다. 만약 배경이 흰색인 데이터라면 `value=(255,255,255)`로 변경해야 모델이 경계선을 오인하지 않습니다.
- **RandomResizedCrop**: `scale=(0.3, 1.0)`은 이미지가 최소 30% 크기까지 줌인(Zoom-in) 될 수 있음을 의미합니다. [cite_start]데이터셋의 물체가 너무 크거나 작다면 이 범위를 조절하세요[cite: 10].